In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE

audio_features_df = pd.read_csv("./preprocessing/audio_features.csv")
bert_embeddings_df = pd.read_csv("./preprocessing/bert_embeddings.csv")

# Merge datasets on Participant_ID
merged_df = pd.merge(audio_features_df, bert_embeddings_df, on="Participant_ID")

# Drop unnecessary columns
merged_df_cleaned = merged_df.drop(columns=["audio_name", "audio_class", "Label_y", "Participant_ID"])

# Extract features (X) and labels (y)
X = merged_df_cleaned.drop(columns=["Label_x"]).values  # Features (Audio + BERT Embeddings)
y = merged_df_cleaned["Label_x"].values  # Labels (Classification target)

# Standardize features (important for CNN-LSTM stability)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Balance the dataset using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X_resampled, dtype=torch.float32).unsqueeze(1)  # (batch, 1, features)
y_tensor = torch.tensor(y_resampled, dtype=torch.long)

# Split into train & test sets (40% for validation)
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.4, random_state=42)

# Create DataLoader for batch training
batch_size = 32
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Compute Class Weights for Imbalanced Data
class_weights = compute_class_weight('balanced', classes=np.unique(y_resampled), y=y_resampled)
class_weights = torch.tensor(class_weights, dtype=torch.float32)

# Define CNN-LSTM Model
class CNN_LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=1):
        super(CNN_LSTM, self).__init__()

        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=5, stride=1, padding=2)  
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, stride=1, padding=2)

        self.adaptive_pool = nn.AdaptiveAvgPool1d(128)  
        self.batch_norm = nn.BatchNorm1d(128)

        self.lstm = nn.LSTM(128, hidden_dim, num_layers=num_layers, bidirectional=True, batch_first=True)

        self.dropout = nn.Dropout(0.6)  
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, x):
        x = self.conv1(x).relu()
        x = self.conv2(x).relu()
        x = self.adaptive_pool(x)
        x = self.batch_norm(x)

        x, _ = self.lstm(x)
        x = self.dropout(x[:, -1, :])
        return self.fc(x)

input_dim = X_train.shape[2]  
hidden_dim = 256  
output_dim = len(np.unique(y_resampled))
num_layers = 2  

model = CNN_LSTM(input_dim, hidden_dim, output_dim, num_layers)

# Define loss function with Class Weights
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)  # LR decay

# Training with Early Stopping
num_epochs = 50
best_loss = float("inf")
patience = 5
counter = 0

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)

    model.eval()
    with torch.no_grad():
        val_outputs = model(X_test)
        val_loss = criterion(val_outputs, y_test)

    scheduler.step(val_loss)  # Reduce learning rate if needed

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}, Val Loss: {val_loss:.4f}")

    if val_loss < best_loss:
        best_loss = val_loss
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered.")
            break

model.eval()
y_pred_list = []

with torch.no_grad():
    for batch_X, _ in test_loader:
        y_pred = model(batch_X)
        y_pred_classes = torch.argmax(y_pred, dim=1).cpu().numpy()
        y_pred_list.extend(y_pred_classes)

accuracy = accuracy_score(y_test.numpy(), y_pred_list)
print(f"Test Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(classification_report(y_test.numpy(), y_pred_list))


c:\Users\kakol\miniconda3\envs\my_env\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch [1/50], Loss: 0.6974, Val Loss: 0.6872
Epoch [2/50], Loss: 0.6504, Val Loss: 0.6837
Epoch [3/50], Loss: 0.5829, Val Loss: 0.6738
Epoch [4/50], Loss: 0.4992, Val Loss: 0.6551
Epoch [5/50], Loss: 0.4133, Val Loss: 0.6224
Epoch [6/50], Loss: 0.3279, Val Loss: 0.5713
Epoch [7/50], Loss: 0.2651, Val Loss: 0.5072
Epoch [8/50], Loss: 0.1958, Val Loss: 0.4544
Epoch [9/50], Loss: 0.1390, Val Loss: 0.4251
Epoch [10/50], Loss: 0.0782, Val Loss: 0.4057
Epoch [11/50], Loss: 0.0423, Val Loss: 0.3797
Epoch [12/50], Loss: 0.0141, Val Loss: 0.3500
Epoch [13/50], Loss: 0.0079, Val Loss: 0.3910
Epoch [14/50], Loss: 0.0046, Val Loss: 0.4558
Epoch [15/50], Loss: 0.0020, Val Loss: 0.3088
Epoch [16/50], Loss: 0.0007, Val Loss: 0.3873
Epoch [17/50], Loss: 0.0007, Val Loss: 0.4514
Epoch [18/50], Loss: 0.0004, Val Loss: 0.5207
Epoch [19/50], Loss: 0.0002, Val Loss: 0.5879
Epoch [20/50], Loss: 0.0002, Val Loss: 0.6255
Early stopping triggered.
Test Accuracy: 0.8500
Classification Report:
              prec